In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np

from src.spectra.gen_spectra import gen_spectrum
from src.file_io import mzML
from src.utils import insort_by_index
from pyteomics import fasta
from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections

# # Dimension of our vector space
# dimension = 500

# # Create a random binary hash with 10 bits
# rbp = RandomBinaryProjections('rbp', 10)

# # Create engine with pipeline configuration
# engine = Engine(dimension, lshashes=[rbp])

# # Index 1000000 random vectors (set their data to a unique string)
# for index in range(100000):
#     v = numpy.random.randn(dimension)
#     engine.store_vector(v, 'data_%d' % index)

# # Create random query vector
# query = numpy.random.randn(dimension)

# # Get nearest neighbours
# N = engine.neighbours(query)

In [2]:
spectra_file = '../data/testing_output/cis_spliced/realisticCisSplicedSpectra.mzML'
fasta_file = '../../testing framework/data/databases/100prots.fasta'

prot_iterator = fasta.read(fasta_file)
spectra_iterator = mzML.read(spectra_file)

In [3]:
dimension = 50
rbp = RandomBinaryProjections('rbp', 10)
engine = Engine(dimension, lshashes=[rbp])

prot_tracker = {}

# run all of our kmer b and y spectra through
for i, p in enumerate(prot_iterator):
    
    print(f'on protein {i}/100\r', end = '')
    
    prot_tracker[i] = p
    
    for j in range(len(p.sequence) - 3):
        for k in range(3, 25):
            kmer_len = k if j + k < len(p.sequence) else len(p.sequence) - (k + j)
            kmer = p.sequence[j:j+kmer_len]

            spec_b = gen_spectrum(kmer, ion='b')['spectrum'] 
            spec_b += [0 for _ in range(dimension - len(spec_b))]

            spec_y = gen_spectrum(kmer, ion='y')['spectrum'] 
            spec_y += [0 for _ in range(dimension - len(spec_y))]

            engine.store_vector(spec_b, f'{kmer}?p_{i}')
            engine.store_vector(spec_y, f'{kmer}?p_{i}')

            if kmer == 'MKHYEVEI':
                print(spec_b)
                print(spec_y)


[132.04776143499998, 260.142724435, 397.20163643499995, 560.2649564349999, 689.3075494349999, 788.3759634349999, 917.4185564349999, 1030.502620435, 66.52751893499999, 130.575000435, 199.10445643499997, 280.63611643499996, 345.15741293499997, 394.69161993499995, 459.21291643499995, 515.754948435, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[132.101905135, 261.14449813500005, 360.21291213500007, 489.2555051350001, 652.3188251350001, 789.3777371350001, 917.4727001350001, 1048.5131851350002, 66.554590785, 131.07588728500002, 180.61009428500003, 245.13139078500004, 326.66305078500005, 395.19250678500003, 459.23998828500004, 524.760230785, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
print(prot_tracker[0])

Protein(description='sp|Q9CY27|TECR_MOUSE Very-long-chain enoyl-CoA reductase OS=Mus musculus OX=10090 GN=Tecr PE=1 SV=1', sequence='MKHYEVEIRDAKTREKLCFLDKVEPQATISEIKTLFTKTHPQWYPARQSLRLDPKGKSLKDEDVLQKLPVGTTATLYFRDLGAQISWVTVFLTEYAGPLFIYLLFYFRVPFIYGRKYDFTSSRHTVVHLACMCHSFHYIKRLLETLFVHRFSHGTMPLRNIFKNCTYYWGFAAWMAYYINHPLYTPPTYGVQQVKLALAVFVICQLGNFSIHMALRDLRPAGSKTRKIPYPTKNPFTWLFLLVSCPNYTYEVGSWIGFAILTQCVPVALFSLVGFTQMTIWAKGKHRSYLKEFRDYPPLRMPIIPFLL')


In [5]:
testspec = gen_spectrum('MKHYEVEI')['spectrum']
testspec += [0 for _ in range(dimension - len(testspec))]

print(engine.neighbours(testspec))

[]
